In [39]:
from math import pi
from CoolProp.CoolProp import PropsSI

In [40]:
def circular_area(diameter):
    return diameter ** 2 * pi / 4


def outlet_enthalpy_isentropic_expansion(h_1, p_1, p_2, eta, wf):
    h_2s = PropsSI("H", "P", p_2, "S", PropsSI("S", "H", h_1, "P", p_1, wf), wf)
    return eta * (h_2s - h_1) + h_1


def sound_speed(p, h, wf):
    Q = PropsSI("Q", "P", p, "H", h, wf)
    if Q > 0 and Q < 1:
        c_l = PropsSI("A", "P", p, "Q", 0, wf)
        c_g = PropsSI("A", "P", p, "Q", 1, wf)
        rho_l = PropsSI("D", "P", p, "Q", 0, wf)
        rho_g = PropsSI("D", "P", p, "Q", 1, wf)

        return (
            1 / (
                (1 - Q) * ((1 - Q) / (c_l ** 2) + (Q * rho_l) / (rho_g * c_g ** 2)) ** 0.5
                + Q * ((Q / c_g ** 2) + ((1 - Q) * rho_g) / (rho_l * c_l ** 2)) ** 0.5
            )
        ) ** 0.5

    else:
        return PropsSI("A", "P", p, "H", h, wf)


D_t = 2.82 * 1e-3
D_p1 = 5.10 * 1e-3
D_3 = 9.20 * 1e-3
P_p = 6.04 * 1e5
P_s = 0.40 * 1e5
omega_star = 0.438
eta_n = 0.95

wf = "R141b"

A_t = circular_area(D_t)
A_p1 = circular_area(D_p1)
A_3 = circular_area(D_3)

# assume saturated states
T_p = PropsSI("T", "P", P_p, "Q", 0, wf)
T_s = PropsSI("T", "P", P_s, "Q", 1, wf)
h_p = PropsSI("H", "P", P_p, "Q", 0, wf)
h_s = PropsSI("H", "P", P_s, "Q", 1, wf)

P_t = P_p * 0.8
d = 1e-3

while True:

    h_t = outlet_enthalpy_isentropic_expansion(h_p, P_p, P_t, eta_n, wf)
    c_t = sound_speed(P_t, h_t, wf)
    h_t_2 = h_p - c_t ** 2 / 2
    residual = h_t - h_t_2

    P_t += d

    h_t = outlet_enthalpy_isentropic_expansion(h_p, P_p, P_t, eta_n, wf)
    c_t = sound_speed(P_t, h_t, wf)
    h_t_2 = h_p - c_t ** 2 / 2
    residual_u = h_t - h_t_2

    derivative = (residual_u - residual) / (d)
    P_t -= d

    P_t -= residual / derivative
    if abs(residual) < 1e-6:
        break

h_t = outlet_enthalpy_isentropic_expansion(h_p, P_p, P_t, eta_n, wf)
rho_t = PropsSI("D", "P", P_t, "H", h_t, wf)
c_t = (2 * (h_p - h_t)) ** 0.5
m = rho_t * A_t * c_t
m


0.03792447371587068

In [27]:
import numpy as np

P_p1 = P_t * np.linspace(0.1, 0.3)

# while True:
h_p1 = outlet_enthalpy_isentropic_expansion(h_t, P_t, P_p1, eta_n, wf)
rho_p1 = PropsSI("D", "P", P_p1, "H", h_p1, wf)
c_p1 = m / (A_p1 * rho_p1)
h_p1_2 = h_t + c_t ** 2 / 2 - c_p1 ** 2 / 2

residual = h_p1 - h_p1_2

residual

array([19499.09727459, 17116.81539665, 15032.05075435, 13200.47755115,
       11585.56539015, 10157.00346866,  8889.48275353,  7761.74633838,
        6755.84271764,  5856.53402476,  5050.82364361,  4327.57652527,
        3677.21205014,  3091.45406726,  2563.12630342,  2085.98400258,
        1654.57467206,  1264.122346  ,   910.43095288,   589.80328199,
         298.97274877,    35.04571073,  -204.54748104,  -422.09416695,
        -619.63500027,  -798.99448666,  -961.80724926, -1109.54068553,
       -1243.51456798, -1364.918048  , -1474.82444613, -1574.2041498 ,
       -1663.93588838, -1744.81661309, -1817.57017405, -1882.85495775,
       -1941.27062376, -1993.36405894, -2039.63465061, -2080.53896541,
       -2116.49490842, -2147.88542702, -2175.06181492, -2198.34666445,
       -2218.03650883, -2234.40419078, -2247.70098885, -2258.15852924,
       -2265.99050715, -2271.39423874])

In [28]:
P_p1 = P_t * 0.1

while True:
    h_p1 = outlet_enthalpy_isentropic_expansion(h_t, P_t, P_p1, eta_n, wf)
    rho_p1 = PropsSI("D", "P", P_p1, "H", h_p1, wf)
    c_p1 = m / (A_p1 * rho_p1)
    h_p1_2 = h_t + c_t ** 2 / 2 - c_p1 ** 2 / 2

    residual = h_p1 - h_p1_2

    P_p1 += d

    h_p1 = outlet_enthalpy_isentropic_expansion(h_t, P_t, P_p1, eta_n, wf)

    rho_p1 = PropsSI("D", "P", P_p1, "H", h_p1, wf)
    c_p1 = m / (A_p1 * rho_p1)
    h_p1_2 = h_t + c_t ** 2 / 2 - c_p1 ** 2 / 2

    P_p1 -= d

    derivative = ((h_p1 - h_p1_2) - residual) / d

    P_p1 -= residual / derivative

    if abs(residual) <= 1e-6:
        break


# recalculate to get true values
h_p1 = outlet_enthalpy_isentropic_expansion(h_t, P_t, P_p1, eta_n, wf)
rho_p1 = PropsSI("D", "P", P_p1, "H", h_p1, wf)
c_p1 = m / (A_p1 * rho_p1)

P_p, P_t, P_p1

(604000.0, 552101.9918129267, 102849.55832729701)

In [130]:
# P_s2 = P_p2 = P_2
eta_s = 0.85

P_2 = np.linspace(0.1, 1.1, 200) * P_p1
for P_2 in P_2:

    h_s2 = outlet_enthalpy_isentropic_expansion(h_s, P_s, P_2, eta_s, wf)
    rho_s2 = PropsSI("D", "P", P_2, "H", h_s2, wf)

    c_s2 = sound_speed(P_2, h_s2, wf)
    m_s = omega_star * m
    A_s2 = m_s / (c_s2 * rho_s2)

    h_p2 = outlet_enthalpy_isentropic_expansion(h_p1, P_p1, P_2, eta_n, wf)
    rho_p2 = PropsSI("D", "P", P_2, "H", h_p2, wf)

    c_p2 = (((m * (h_p1 + c_p1 ** 2 / 2) + m_s * h_s - m_s * (h_s2 + c_s2 ** 2 / 2)) / m - h_p2) * 2) ** 0.5
    A_p2 = m / (rho_p2 * c_p2)
    print(A_s2 + A_p2 - A_3)

0.0029592394041992806
0.0027912448435439643
0.0026406598528792855
0.0025049833154258995
0.002382162919645794
0.0022704999531146164
0.0021685771618817637
0.002075203443547265
0.0019893709875188413
0.0019102217302532027
0.0018370208592019543
0.0017691357056058283
0.0017060187966170468
0.0016471941463798191
0.00159224609034077
0.0015408101320419526
0.0014925653940263722
0.0014472283561124243
0.001404547633496769
0.0013642995998551012
0.001326284701047509
0.0012903243363235838
0.0012562582082566711
0.0012239420617228098
0.0011932457472825246
0.0011640515562589657
0.0011362527843255445
0.0011097524880512917
0.0010844624050075635
0.0010603020130248017
0.0010371977082461888
0.001015082084941689
0.0009938933027693988
0.0009735745294159677
0.000954073448405483
0.0009353418234098626
0.0009173351116801903
0.0009000121202951861
0.0008833346998265673
0.0008672674707820095
0.0008517775788296055
0.0008368344753520709
0.0008224097203423771
0.0008084768050464214
0.0007950109920958734
0.0007819891711627